# Position tracking

The classes for object (arena, bridge, mouse lever) detections are in the autopipy package.

There are a few steps in this procedure to go from the raw video of the arena to the position of the animal. 

Steps with the `arena_top.avi` file

* Detect the bridge and arena in arena_top.avi
* Crop the arena_top.avi to 480x480

Steps with the cropped video

* Detect arena in cropped video
* Detect bridge in cropped video
* Detect the mouse and lever in cropped video

There is a jupyter notebook in `autopi_analysis/kevin/autopipy_dev_notebooks` in which all the steps for a single session are performed separately. This is a good place to go if you have problems running the analysis on one session.

Below is the code to run the analysis on several sessions. 

It takes approximately **30 minutes per session** with a decent GPU. You can of course run different sessions on different GPU to speed up the process.

You will need to run this code from a DLC-GPU virtual environment.

In [3]:
import os
os.environ["DLClight"]="True" # if you don't need dlb gui
import numpy as np
import pandas as pd
import os.path
import autopipy
import importlib
from autopipy.project import Project
from autopipy.video_utilities import positionTrackingFromArenaTopVideo

Let's get our project with the list of sessions

In [5]:
projectName="autopi_behavior_2021"
dataPath="/adata/projects/autopi_behavior_2021"
dlcModelPath="/adata/models"
bridgeModel1 ="bridgeDetection_640_480-Allen-2021-02-10"
mouseLeverModel ="arena_top-Allen-2019-10-30"
bridgeModel2 = "bridgeDetection_480_480-Allen-2021-01-23"
arenaMinRadius= 190
arenaMaxRadius= 230


myProject = Project(name=projectName,dataPath=dataPath,dlcModelPath=dlcModelPath)

# read a file with the session names
fn=myProject.dataPath+"/memory_sessions"
print("Reading " + fn)
sessionNamesTesting = pd.read_csv(fn) # this will be a pandas dataframe

# create a list of session object in the project object
myProject.createSessionList(sessionNameList=sessionNamesTesting.sessionName.to_list())
print("We have {} recording sessions".format(len(myProject.sessionList)))
sessionNamesTesting

Project name: autopi_behavior_2021
dataPath: /adata/projects/autopi_behavior_2021
dlcModelPath: /adata/models
Reading /adata/projects/autopi_behavior_2021/memory_sessions
We have 10 recording sessions


,sessionName
0,mn9686-12052021-1702
1,TYY5622-12052021-1805
2,TYY5630-12052021-1909
3,mn9686-14052021-1501
4,TYY5622-14052021-1609
5,TYY5630-14052021-1818
6,TYY9524-14052021-2029
7,mn9686-16052021-1443
8,TYY5630-16052021-1810
9,TYY5622-16052021-1621


The function doing all the video processing and object tracking is in the file `video_utilities.py` of the `autopipy` package. 
The function is called `positionTrackingFromArenaTopVideo`.
It relies on a 3 trained deeplabcut models to detect 

* 1) the bridge in 680x480 videos
* 2) the mouse and lever in 480x480 videos
* 3) the bridge in 480x480 videos

You can set the path where these models are found and the name of the deeplabcut models via the function arguments.


## Process a single session

Let's start with a single session and make sure this works fine.

In [4]:
importlib.reload(autopipy.video_utilities)
from autopipy.video_utilities import positionTrackingFromArenaTopVideo
from autopipy.dlcObjectDetectors import MouseLeverDetector
ses = myProject.sessionList[-1]
positionTrackingFromArenaTopVideo(ses,modelDir=dlcModelPath)


Startint at 20:39:56
labelImage: /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/arenaDetection.png
/tmp/tmpVid.avi created for bridge detection
Running dlc.analyze_video on /tmp/tmpVid.avi
Using snapshot-460000 for model /adata/models/bridgeDetection_640_480-Allen-2021-02-10/dlc-models/iteration-0/bridgeDetection_640_480Feb10-trainset95shuffle1
Initializing ResNet


  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


510it [00:08, 61.68it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Bridge coordinates: [[273   0]
 [273  15]
 [320  15]
 [320   0]]
labelImage: /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/bridgeDetection.png
Time elapsed 0:00:20.604771
Cropping and masking 109478 frames in /adata/pr

...

Saving results in /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/TYY5630-12052021-1909.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/TYY5630-12052021-1909.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
Saving position data to /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/TYY5630-12052021-1909.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.csv
dlc.loadPositionData() read data from /adata

  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  480 480
Starting to extract posture


510it [00:05, 85.12it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Bridge coordinates: [[204   0]
 [204  12]
 [255  12]
 [255   0]]
labelImage: /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-12052021-1909/bridgeDetectionCropped.png
labelImage: /adata/projects/autopi_behavior_2021/TYY5630/TYY5630-120

In [19]:
importlib.reload(autopipy.video_utilities)
from autopipy.video_utilities import positionTrackingFromArenaTopVideo

ses = myProject.sessionList[-1]
positionTrackingFromArenaTopVideo(ses=ses,modelDir=myProject.dlcModelPath,
                                  bridge640_480Model = bridgeModel1,
                                  mouseLeverModel = mouseLeverModel,
                                  bridge480_480Model = bridgeModel2,
                                 arenaMinRadius=arenaMinRadius , arenaMaxRadius=arenaMaxRadius)


Startint at 18:06:10
labelImage: /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/arenaDetection.png
/tmp/tmpVid.avi created for bridge detection
Running dlc.analyze_video on /tmp/tmpVid.avi
Using snapshot-460000 for model /adata/models/bridgeDetection_640_480-Allen-2021-02-10/dlc-models/iteration-0/bridgeDetection_640_480Feb10-trainset95shuffle1
Initializing ResNet


  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


510it [00:08, 59.19it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Bridge coordinates: [[287   0]
 [287  29]
 [333  29]
 [333   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/bridgeDetection.png
Time elapsed 0:00:20.019249
Cropping and masking 109687 frames in /adata/proj

...

Saving results in /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/mn3246-05032021-1534.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/mn3246-05032021-1534.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
Saving position data to /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/mn3246-05032021-1534.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.csv
dlc.loadPositionData() read data from /adata/projects/a

  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  480 480
Starting to extract posture


510it [00:05, 86.99it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Bridge coordinates: [[206   0]
 [206  31]
 [256  31]
 [256   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn3246/mn3246-05032021-1534/bridgeDetectionCropped.png
labelImage: /adata/projects/autopi_behavior_2021/mn3246/mn3246-0503202

In [6]:
myProject.getSession("mn5183-13082020-1011")

IndexError: list index out of range

In [4]:
## Sessions to re-run with new network
sesNameList = ["mn7701-19122019-1216","mn7711-19122019-1429"]
for sesName in sesNameList:
    ses=myProject.getSession(sesName)
    positionTrackingFromArenaTopVideo(ses=ses,modelDir=myProject.dlcModelPath)

Startint at 10:15:52
labelImage: /adata/projects/autopi_behavior_2021/mn7701/mn7701-19122019-1216/arenaDetection.png
/tmp/tmpVid.avi created for bridge detection
Running dlc.analyze_video on /tmp/tmpVid.avi
Using snapshot-460000 for model /adata/models/bridgeDetection_640_480-Allen-2021-02-10/dlc-models/iteration-0/bridgeDetection_640_480Feb10-trainset95shuffle1
Initializing ResNet


  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


510it [00:08, 56.88it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Bridge coordinates: [[293   0]
 [293  30]
 [335  30]
 [335   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn7701/mn7701-19122019-1216/bridgeDetection.png
Time elapsed 0:00:21.893074
Cropping and masking 106253 frames in /adata/proj

...

Saving results in /adata/projects/autopi_behavior_2021/mn7711/mn7711-19122019-1429...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/mn7711/mn7711-19122019-1429/mn7711-19122019-1429.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/mn7711/mn7711-19122019-1429/mn7711-19122019-1429.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
Saving position data to /adata/projects/autopi_behavior_2021/mn7711/mn7711-19122019-1429/mn7711-19122019-1429.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.csv
dlc.loadPositionData() read data from /adata/projects/a

  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  480 480
Starting to extract posture


510it [00:05, 85.07it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Bridge coordinates: [[218   0]
 [218  29]
 [260  29]
 [260   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn7711/mn7711-19122019-1429/bridgeDetectionCropped.png
labelImage: /adata/projects/autopi_behavior_2021/mn7711/mn7711-1912201

## Group of sessions
And to run a group of sessions, simply run this in a for loop. I will save the session name in a file to keep track of what is done.

pc73 from home.
Friday: I did 0:10 as a first try and worked great. 5%
Friday: I tried 10:40 overnight to test overnight vpn connection, manage to do 32 sessions. 

It is too slow if run only on one computer.
pc59 and pc 73

* pc73: [32:100]
* pc59: [100:]

In [6]:
for ses in myProject.sessionList[7:] :
    positionTrackingFromArenaTopVideo(ses=ses,modelDir=myProject.dlcModelPath)
   # text_file = open(myProject.dataPath+"/results/dlcDone", "a")
   # text_file.write("{}\n".format(ses.name))
   # text_file.close()

Startint at 21:17:46
labelImage: /adata/projects/autopi_behavior_2021/mn9686/mn9686-16052021-1443/arenaDetection.png
/tmp/tmpVid.avi created for bridge detection
Running dlc.analyze_video on /tmp/tmpVid.avi
Using snapshot-460000 for model /adata/models/bridgeDetection_640_480-Allen-2021-02-10/dlc-models/iteration-0/bridgeDetection_640_480Feb10-trainset95shuffle1
Initializing ResNet


  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  640 480
Starting to extract posture


510it [00:08, 59.24it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_640_480Feb10shuffle1_460000.h5
Bridge coordinates: [[270   0]
 [270  22]
 [323  22]
 [323   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn9686/mn9686-16052021-1443/bridgeDetection.png
Time elapsed 0:00:20.049256
Cropping and masking 157232 frames in /adata/proj

...

Saving results in /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-16052021-1621...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-16052021-1621/TYY5622-16052021-1621.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-16052021-1621/TYY5622-16052021-1621.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.h5
Saving position data to /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-16052021-1621/TYY5622-16052021-1621.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_950000.csv
dlc.loadPositionData() read data from /adata

  0%|          | 0/500 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  480 480
Starting to extract posture


510it [00:05, 90.44it/s]                         


Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_320000.h5
Bridge coordinates: [[213   0]
 [213  23]
 [264  23]
 [264   0]]
labelImage: /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-16052021-1621/bridgeDetectionCropped.png
labelImage: /adata/projects/autopi_behavior_2021/TYY5622/TYY5622-160

In [ ]:
for ses in myProject.sessionList[100:] :
    positionTrackingFromArenaTopVideo(ses=ses,modelDir=myProject.dlcModelPath)
    text_file = open(myProject.dataPath+"/results/dlcDone", "a")
    text_file.write("{}\n".format(ses.name))
    text_file.close()

## Check that all went fine

First we will make sure that we have all session names in the `dlcDone` file.

In [3]:
done = pd.read_csv(myProject.dataPath+"/results/dlcDone",header=None)
sessionNamesTesting
print("Number of sessions: {}, processed sessions: {}".format(len(sessionNamesTesting),len(done)))
if len(sessionNamesTesting) > len(done) :
    print("Missing sessions are:")
    print(sessionNamesTesting.sessionName[~sessionNamesTesting.sessionName.isin(done[0])])

if np.all(sessionNamesTesting.sessionName.isin(done[0])) :
    print("All sessions processed")

Number of sessions: 187, processed sessions: 188
All sessions processed


## Check bridge and arena detection

The bridge and arena coordinates are critical for the subsequent analysis. We will make a folder containing an image of the detection for each session. One can then quickly scroll through the images to identify any problematic case.

In [13]:
# create a directory for the images
directory = myProject.dataPath + "/results/arenaBridgeDetectionImages"
try:
    os.stat(directory)
except:
    os.mkdir(directory)       

In [14]:
from shutil import copyfile

for ses in myProject.sessionList:
    src = ses.path + "/arenaBridgeDetectionCropped.png"
    dst = directory + "/" + ses.name + ".png"
    if not os.path.isfile(src):
        print (src + " does not exist")
        break
    try:
        copyfile(src, dst)
    except:
        print("error copying the file for " + ses.name )


# Rerun problematic sessions

I put the sessions with problems in a file in the `results/redo`

The main problem was the arena detection that was not perfect.

In [15]:
projectName="autopi_behavior_2021"
dataPath="/adata/projects/autopi_behavior_2021"
dlcModelPath="/adata/models"
bridgeModel1 ="bridgeDetection_640_480-Allen-2021-02-10"
mouseLeverModel ="arena_top-Allen-2019-10-30"
bridgeModel2 = "bridgeDetection_480_480-Allen-2021-01-23"
arenaMinRadius= 190
arenaMaxRadius= 230

myProject = Project(name=projectName,dataPath=dataPath,dlcModelPath=dlcModelPath)

# read a file with the session names
fn=myProject.dataPath+"/redo"
print("Reading " + fn)
sessionNamesTesting = pd.read_csv(fn) # this will be a pandas dataframe

# create a list of session object in the project object
myProject.createSessionList(sessionNameList=sessionNamesTesting.sessionName.to_list())
print("We have {} recording sessions".format(len(myProject.sessionList)))

Project name: autopi_behavior_2021
dataPath: /adata/projects/autopi_behavior_2021
dlcModelPath: /adata/models
Reading /adata/projects/autopi_behavior_2021/redo
We have 4 recording sessions


In [3]:
directory = myProject.dataPath + "/results/arenaDetectionImages"
try:
    os.stat(directory)
except:
    os.mkdir(directory)   

In [17]:
from autopipy.cvObjectDetectors import ArenaDetector
from datetime import datetime

for ses in myProject.sessionList :
    positionTrackingFromArenaTopVideo(ses=ses,modelDir=myProject.dlcModelPath,
                                      bridge640_480Model = bridgeModel1,
                                      mouseLeverModel = mouseLeverModel,
                                      bridge480_480Model = bridgeModel2,
                                      arenaMinRadius=arenaMinRadius , arenaMaxRadius=arenaMaxRadius,
                                      arenaCircleMethod="median")
   

TypeError: positionTrackingFromArenaTopVideo() got an unexpected keyword argument 'arenaCircleMethod'

In [23]:
from autopipy.cvObjectDetectors import ArenaDetector
from autopipy.dlcObjectDetectors import BridgeDetector
from autopipy.dlcObjectDetectors import MouseLeverDetector
from autopipy.video_utilities import arenaBridgeDetectionImage
from datetime import datetime

modelDir="/adata/models"
bridge640_480Model = "bridgeDetection_640_480-Allen-2021-02-10"
mouseLeverModel = "arena_top-Allen-2019-10-30"
bridge480_480Model = "bridgeDetection_480_480-Allen-2021-01-23"
arenaMinRadius= 180
arenaMaxRadius= 230
arenaCircleMethod = "min"
numFramesArenaDetection=500
numFramesBridgeDetection=500

print(modelDir)
print(mouseLeverModel)
configFile=modelDir+"/"+ mouseLeverModel + "/config.yaml"


for ses in [myProject.sessionList[0]] :
    
    videoFile=ses.fileNames["arena_top.avi"]
    croppedVideoFile = os.path.splitext(videoFile)[0]+".cropped.avi"
    arenaImageFile=ses.path+"/arenaDetectionCropped.png"
    bridge480_480Model=bridgeModel2
    modelDir = dlcModelPath
    
    
    arenaD = ArenaDetector()
    aCoord = arenaD.detectArenaCoordinates(pathVideoFile=croppedVideoFile, minRadius=arenaMinRadius, 
                                  maxRadius=arenaMaxRadius, numFrames=100, blur=11, circleMethod='median')
    arenaD.labelImage(pathVideoFile=croppedVideoFile,outputImageFile=arenaImageFile)
    np.savetxt(ses.fileNames["arenaCoordinates"],aCoord,delimiter=",") 


    configFile=modelDir+"/"+ mouseLeverModel + "/config.yaml"
    if not os.path.isfile(configFile):
        print(configFile+ " is missing")
    mouseLeverD = MouseLeverDetector(pathConfigFile=configFile)
    # save position data to file
    mouseLeverD.savePositionOrientationToFile(pathVideoFile=croppedVideoFile, 
                                              fileName = ses.fileNames["mouseLeverPosition.csv"])
    
    
    
    configFile = modelDir + "/" + bridge480_480Model + "/config.yaml"
    if not os.path.isfile(configFile):
        print(configFile+ " is missing")
    bridgeImageFile = ses.path+"/bridgeDetectionCropped.png"
    bridgeD = BridgeDetector(pathConfigFile=configFile)
    bCoord = bridgeD.detectBridgeCoordinates(pathVideoFile=croppedVideoFile,numFrames=100, skip=30)
    bridgeD.labelImage(pathVideoFile=videoFile,outputImageFile=bridgeImageFile)
    np.savetxt(ses.fileNames["bridgeCoordinates"],bCoord,delimiter=",")
    
    outputImageFile=ses.path+"/arenaBridgeDetectionCropped.png"
    arenaBridgeDetectionImage(pathVideoFile=croppedVideoFile,
                              outputImageFile=outputImageFile,
                              arenaCoordinates = aCoord,
                              bridgeCoordinates = bCoord)

/adata/models
arena_top-Allen-2019-10-30
labelImage: /adata/projects/autopi_behavior_2021/mn4656/mn4656-30092019-2001/arenaDetectionCropped.png
dlc.loadPositionData() read data from /adata/projects/autopi_behavior_2021/mn4656/mn4656-30092019-2001/mn4656-30092019-2001.arena_top.croppedDLC_resnet_50_arena_topOct30shuffle1_1030000.h5
/tmp/tmpVid.avi created for bridge detection
Running dlc.analyze_video on /tmp/tmpVid.avi
Using snapshot-420000 for model /adata/models/bridgeDetection_480_480-Allen-2021-01-23/dlc-models/iteration-0/bridgeDetection_480_480Jan23-trainset90shuffle1
Initializing ResNet


  0%|          | 0/100 [00:00<?, ?it/s]

Starting to analyze %  /tmp/tmpVid.avi
/tmp  already exists!
Loading  /tmp/tmpVid.avi
Duration of video [s]:  3.33 , recorded with  30.0 fps!
Overall # of frames:  100  found with (before cropping) frame dimensions:  480 480
Starting to extract posture


110it [00:01, 59.72it/s]                         


Detected frames:  100
Saving results in /tmp...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_420000.h5
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_420000.h5
Saving position data to /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_420000.csv
dlc.loadPositionData() read data from /tmp/tmpVidDLC_resnet_50_bridgeDetection_480_480Jan23shuffle1_420000.h5
Bridge coordinates: [[220   0]
 [220  24]
 [259  24]
 [259   0]]
labelImage: /adata/projects/autopi_behavior_2021/mn4656/mn4656-30092019-2001/bridgeDetectionCropped.png
labelImage: /adata/projects/autopi_behavior_2021